In [1]:
import pygame

# Configuración de la ventana
window_width = 1200
window_height = 800

# Configuración de la cuadrícula
grid_size = 10
grid_color = (200, 200, 200)  # Color de la cuadrícula en formato RGB

# Configuración de los elementos adicionales
road_width = 200
sidewalk_width = 100
green_color = (0, 255, 0)
white_color = (255, 255, 255)
grey_color = (128, 128, 128)
yellow_color = (255, 255, 0)

# Inicializar pygame y la ventana
pygame.init()
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Cuadrícula con elementos adicionales")

def draw_grid():
    # Dibujar líneas verticales
    for x in range(0, window_width, grid_size):
        pygame.draw.line(window, grid_color, (x, 0), (x, window_height))

    # Dibujar líneas horizontales
    for y in range(0, window_height, grid_size):
        pygame.draw.line(window, grid_color, (0, y), (window_width, y))

def draw_elements():
    # Dibujar los elementos adicionales
    pygame.draw.rect(window, white_color, (window_width/2 - road_width/2 - sidewalk_width, 0, sidewalk_width, window_height))
    pygame.draw.rect(window, white_color, (window_width/2 + road_width/2, 0, sidewalk_width, window_height))
    pygame.draw.rect(window, white_color, (0, window_height/2 - road_width/2 - sidewalk_width, window_width, sidewalk_width))
    pygame.draw.rect(window, white_color, (0, window_height/2 + road_width/2, window_width, sidewalk_width))

    pygame.draw.rect(window, grey_color, (window_width/2 - road_width/2, 0, road_width, window_height))
    pygame.draw.rect(window, grey_color, (0, window_height/2 - road_width/2, window_width, road_width))

    for y in range(0, window_height, 80):
        pygame.draw.rect(window, yellow_color, (window_width/2 - 5, y, 10, 40))
    for x in range(0, window_width, 80):
        pygame.draw.rect(window, yellow_color, (x, window_height/2 - 5, 40, 10))

# Bucle principal del programa
running = True
while running:
    # Manejo de eventos
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Obtener las coordenadas del cursor relativas a la cuadrícula
    mouse_x, mouse_y = pygame.mouse.get_pos()
    grid_x = (mouse_x // grid_size) * grid_size
    grid_y = (mouse_y // grid_size) * grid_size

    # Rellenar la ventana con color verde
    window.fill(green_color)

    # Dibujar los elementos adicionales
    draw_elements()

    # Dibujar la cuadrícula
    draw_grid()

    # Mostrar las coordenadas del cursor en la cuadrícula
    font = pygame.font.Font(None, 30)
    text = font.render(f"({grid_x}, {grid_y})", True, white_color)
    window.blit(text, (10, 10))

    # Actualizar la ventana
    pygame.display.flip()

# Cerrar pygame al salir
pygame.quit()


pygame 2.4.0 (SDL 2.26.4, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame

class Semaforo:
    LIGHT_COLOR = (255, 0, 0)  # Rojo
    COLLIDER_COLOR = (0, 0, 255, 100)  # Azul transparente

    def __init__(self, x, y, collider_size, light_size):
        self.color = Semaforo.LIGHT_COLOR
        self.collider_rect = pygame.Rect(x, y, collider_size[0], collider_size[1])
        self.light_rect = pygame.Rect(x, y, light_size[0], light_size[1])

    def change_color(self):
        if self.color == Semaforo.LIGHT_COLOR:
            self.color = (0, 255, 0)  # Verde
        else:
            self.color = Semaforo.LIGHT_COLOR

    def is_red(self):
        return self.color == Semaforo.LIGHT_COLOR

    def draw(self, screen):
        pygame.draw.rect(screen, Semaforo.COLLIDER_COLOR, self.collider_rect)
        pygame.draw.rect(screen, self.color, self.light_rect)


# Configuración de pygame
pygame.init()
screen = pygame.display.set_mode((800, 600))
pygame.display.set_caption("Semaforo")
pygame.display.set_alpha(None)
screen.set_alpha(None)

# Crear una instancia de Semaforo
semaforo = Semaforo(100, 100, (10, 10), (20, 40))

# Bucle principal del juego
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill((255, 255, 255))  # Limpiar la pantalla

    # Dibujar el semáforo
    semaforo.draw(screen)

    pygame.display.flip()

pygame.quit()


AttributeError: module 'pygame.display' has no attribute 'set_alpha'

: 